In [1]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('dark_background')

%matplotlib notebook

In [2]:
data = np.load('dataset.npz')

In [3]:
print(data['l'])

[[ 5.36478956  0.6712642 ]
 [ 5.6712674  -0.98397905]
 [ 5.47627721 -2.30056124]
 [ 5.45335227  2.81978672]
 [ 7.26653099  1.75776226]
 [ 7.86526087 -0.54825338]
 [ 7.26342575 -1.90184166]
 [ 9.50045689 -1.84310854]
 [ 8.98894839  0.72538381]
 [ 3.55908128 -1.13565243]
 [ 0.76559652 -1.91577577]
 [-1.26746489 -1.47360298]
 [-1.20953764  0.95077535]
 [ 0.54857082  0.08135548]
 [-0.20345764  2.55330606]
 [ 2.8177184   2.73513504]
 [ 3.38509971  1.36225359]]


In [4]:
X_l = data['l']
X_t = np.concatenate((np.concatenate((data['x_true'], data['y_true']),axis=1), data['th_true']), axis=1)
print(X_t[0])

[ 3.01975613  0.07089905 -2.91015736]


In [5]:
def get_observation_jacobian_1(X_t, X_l, d ):
    # here h matrix is 34 x 3 dimensions right?

    # i will have to see the correction step and find the H dimension that works with the equation
    H = np.zeros((17*2, 3))
    P = X_l[:,0].T - X_t[0] - d*np.cos(X_t[2]) # x
    Q = X_l[:,1].T - X_t[1] - d*np.sin(X_t[2]) # y

    H[::2, 0]  =  np.reciprocal(np.sqrt(P**2 + Q**2)) * P * -1
    H[::2, 1]  =  np.reciprocal(np.sqrt(P**2 + Q**2)) * Q * -1
    H[::2, 2]  =  np.reciprocal(np.sqrt(P**2 + Q**2)) * d * (np.sin(X_t[2])*P - np.cos(X_t[2])*Q)
    H[1::2, 0] =  Q * np.reciprocal((P**2 + Q**2))
    H[1::2, 1] = -P * np.reciprocal((P**2 + Q**2))
    H[1::2, 2] = -d * np.reciprocal((Q**2 + P**2)) * (P*np.cos(X_t[2]) + Q*np.sin(X_t[2])) - 1

    return H

def get_observation_jacobian_2(X_t, X_l, d ):
    H = np.zeros((0, 3))
    for i in range(17):
        p = X_l[i,0].T - X_t[0] - d * np.cos(X_t[2])
        q = X_l[i,1].T - X_t[1] - d * np.sin(X_t[2])
        H_i = np.zeros((2, 3))

        H_i[0,0] = np.reciprocal(np.sqrt(p**2 + q**2)) * p * -1
        H_i[0,1] = np.reciprocal(np.sqrt(p**2 + q**2)) * q * -1
        H_i[0,2] = np.reciprocal(np.sqrt(p**2 + q**2)) * d * (np.sin(X_t[2])*p - np.cos(X_t[2])*q)
        H_i[1,0] =  q / (p**2 + q**2)
        H_i[1,1] = -p / (p**2 + q**2)
        H_i[1,2] = -d * (p*np.cos(X_t[2]) + q*np.sin(X_t[2])) / (p**2 + q**2) -1

        H = np.concatenate(
            (H, H_i),
            axis = 0
        )
    return H

In [14]:
for i in range(len(data['r'])):
    print(data['r'][i])

[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.37430716  2.81830716  4.35230716  4.11630716
  2.26830716  3.90630716  2.69230716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.37330716  2.82230716  4.34230716  4.11030716
  2.27130716  3.90830716  2.69830716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.37330716  2.82030716  4.33130716  4.11830716
  2.27430716  3.90430716  2.69130716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.37330716  2.82230716  4.33630716  4.12430716
  2.26630716  3.90930716  2.69730716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.37230716  2.82530716  4.34830716  4.10630716
  2.26830716  3.90830716  2.69930716  0.        ]


[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.36930716  2.81830716  4.34330716  4.11530716
  2.24930716  3.91130716  2.69030716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.37830716  2.82630716  4.33630716  4.11130716
  2.26930716  3.90230716  2.69630716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.37430716  2.82130716  4.33430716  4.10330716
  2.26630716  3.90730716  2.68930716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.37730716  2.82130716  4.33530716  4.11030716
  2.27730716  3.90430716  2.69330716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.37230716  2.82030716  4.34030716  4.10730716
  2.25530716  3.90930716  2.69530716  0.        ]


[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.43830716  2.70830716  4.21030716  3.99330716
  2.12130716  3.82130716  2.72730716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.43830716  2.70530716  4.19830716  3.99230716
  2.11530716  3.82430716  2.73730716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.43530716  2.70130716  4.21530716  3.99530716
  2.11430716  3.81930716  2.72430716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.43130716  2.70230716  4.19830716  3.99230716
  2.10930716  3.82130716  2.71730716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.43630716  2.71430716  4.21830716  4.00130716
  2.12530716  3.82730716  2.71230716  0.        ]


[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.42830716  2.70230716  4.20430716  3.99330716
  2.10530716  3.83230716  2.71730716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.43430716  2.70530716  4.20430716  3.98930716
  2.11830716  3.83030716  2.71730716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.43330716  2.70430716  4.19730716  3.99230716
  2.11130716  3.82330716  2.71830716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.43530716  2.70730716  4.20830716  3.99430716
  2.12530716  3.82330716  2.72230716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.43430716  2.70930716  4.20930716  3.98930716
  2.10630716  3.82630716  2.72230716  0.        ]


[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.43830716  2.70130716  4.20530716  3.98930716
  2.11230716  3.82230716  2.71430716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.43230716  2.70730716  4.19930716  3.98330716
  2.11430716  3.82330716  2.71630716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.43030716  2.70830716  4.20030716  3.99130716
  2.11730716  3.82430716  2.71630716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.43330716  2.70930716  4.19630716  4.00330716
  2.10230716  3.82030716  2.72030716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.43230716  2.70530716  4.21230716  3.99630716
  2.10930716  3.82430716  2.72430716  0.        ]


...

[ 0.          0.71030716  1.58130716  0.          3.44730716  2.89530716
  2.53830716  0.          4.27530716  1.43330716  0.          0.          0.
  0.          0.          0.          0.        ]
[ 0.          0.69430716  1.55730716  0.          3.44530716  2.90530716
  2.51930716  0.          0.          1.43430716  0.          0.          0.
  0.          0.          0.          0.        ]
[ 0.          0.68630716  1.53030716  0.          3.45030716  2.89430716
  2.50730716  0.          0.          1.43630716  0.          0.          0.
  0.          0.          0.          0.        ]
[ 0.          0.67930716  1.51430716  0.          3.46230716  2.89430716
  2.50130716  0.          0.          1.43830716  4.34830716  0.          0.
  0.          0.          0.          0.        ]
[ 0.          0.67230716  1.49730716  0.          3.46630716  2.88230716
  2.48330716  0.          4.29330716  1.43830716  4.34930716  0.          0.
  0.          0.          0.          0.        ]


[ 1.41330716  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          5.63530716  0.          0.
  0.          2.75630716  1.27730716]
[ 1.40730716  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          5.63530716  0.          0.
  0.          2.73930716  1.25030716]
[ 1.39630716  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          5.63530716  0.          0.
  0.          2.72730716  1.22530716]
[ 1.38830716  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          5.63530716  0.          0.
  0.          2.70230716  1.19230716]
[ 1.36730716  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          5.63530716  0.          0.
  0.          2.67130716  1.18430716]
[ 1.36630716  0.          0.          0.

[ 0.          0.          0.          0.          0.          5.63530716
  0.          0.          0.          3.05430716  0.          0.          0.
  0.          0.          0.          0.73830716]
[ 0.          0.          0.          0.          0.          5.63530716
  0.          0.          0.          3.00630716  0.          0.          0.
  0.          0.          0.          0.73030716]
[ 0.          0.          0.          0.          0.          5.63530716
  0.          0.          0.          2.98630716  0.          0.          0.
  0.          0.          0.          0.72730716]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          2.96130716  0.          0.          0.          0.
  0.          0.          0.72430716]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          2.93130716  0.          0.          0.          0.
  0.          0.          0.72030716]
[ 0.          0.

[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          2.74430716  4.04730716  3.63830716
  1.80830716  3.43530716  2.46830716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          2.73830716  4.05230716  3.64330716
  1.81230716  3.42930716  2.46630716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          2.73330716  4.06630716  3.64030716
  1.82230716  3.43330716  2.46530716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          2.74330716  4.06430716  3.65230716
  1.82030716  3.44130716  2.46830716  0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          2.74730716  4.08030716  3.65930716
  1.83530716  3.43330716  2.47130716  0.        ]


[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.38230716  3.21430716  0.          4.44530716
  2.66430716  4.08730716  2.52930716  1.12430716]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.38030716  3.22130716  0.          4.44230716
  2.68330716  4.10630716  2.53530716  1.11830716]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.38230716  3.21730716  0.          4.43930716
  2.68930716  4.10330716  2.53330716  1.12430716]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.38230716  3.21730716  0.          4.43530716
  2.68230716  4.10130716  2.52530716  1.11530716]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          1.38330716  3.21930716  0.          4.44530716
  2.66430716  4.10230716  2.52730716  1.12430716]


In [11]:
print(H1)

[[-0.96894438 -0.24727878  0.00184345]
 [ 0.09810129 -0.38440296 -0.9603344 ]
 [-0.93620329  0.35145895 -0.05568298]
 [-0.1197001  -0.31885267 -0.97171047]
 [-0.73607801  0.67689672 -0.08276871]
 [-0.19509675 -0.21215412 -0.98382526]
 [-0.6742887  -0.73846784  0.05641141]
 [ 0.19674212 -0.17964355 -0.97800184]
 [-0.93051901 -0.36624361  0.01430413]
 [ 0.0784503  -0.19931951 -0.97880002]
 [-0.9928036   0.11975393 -0.03442852]
 [-0.02405341 -0.19941153 -0.98114224]
 [-0.91220818  0.40972703 -0.06080405]
 [-0.08609907 -0.19168927 -0.98331705]
 [-0.96107376  0.27629193 -0.04893716]
 [-0.04036721 -0.14041622 -0.98725867]
 [-0.9938231  -0.11097585 -0.01199409]
 [ 0.01818015 -0.16280889 -0.98373618]
 [-0.47371236  0.88067962 -0.09658568]
 [-0.65527833 -0.35247034 -0.98072315]
 [ 0.73943022  0.6732332  -0.04856769]
 [-0.2308063   0.2535008  -1.02996831]
 [ 0.93993966  0.34134066 -0.01166414]
 [-0.07657476  0.21086166 -1.0222804 ]
 [ 0.97695194 -0.21345939  0.04318563]
 [ 0.05046989  0.23098846

In [12]:
print(H2)

[[-0.96894438 -0.24727878  0.00184345]
 [ 0.09810129 -0.38440296 -0.9603344 ]
 [-0.93620329  0.35145895 -0.05568298]
 [-0.1197001  -0.31885267 -0.97171047]
 [-0.73607801  0.67689672 -0.08276871]
 [-0.19509675 -0.21215412 -0.98382526]
 [-0.6742887  -0.73846784  0.05641141]
 [ 0.19674212 -0.17964355 -0.97800184]
 [-0.93051901 -0.36624361  0.01430413]
 [ 0.0784503  -0.19931951 -0.97880002]
 [-0.9928036   0.11975393 -0.03442852]
 [-0.02405341 -0.19941153 -0.98114224]
 [-0.91220818  0.40972703 -0.06080405]
 [-0.08609907 -0.19168927 -0.98331705]
 [-0.96107376  0.27629193 -0.04893716]
 [-0.04036721 -0.14041622 -0.98725867]
 [-0.9938231  -0.11097585 -0.01199409]
 [ 0.01818015 -0.16280889 -0.98373618]
 [-0.47371236  0.88067962 -0.09658568]
 [-0.65527833 -0.35247034 -0.98072315]
 [ 0.73943022  0.6732332  -0.04856769]
 [-0.2308063   0.2535008  -1.02996831]
 [ 0.93993966  0.34134066 -0.01166414]
 [-0.07657476  0.21086166 -1.0222804 ]
 [ 0.97695194 -0.21345939  0.04318563]
 [ 0.05046989  0.23098846